In [1]:
import numpy as np
import numba
import datetime
from quantecon.markov.approximation import std_norm_cdf
from scipy.linalg import solve
import matplotlib.pyplot as plt
from interpolation import interp

In [2]:
#z_first=qe.markov.approximation.tauchen(rho=0.96,sigma_u=0.045**0.5,m=4,n=18)
#z_second=qe.markov.approximation.tauchen(rho=0.96,sigma_u=0.045**0.5,m=6,n=2)


def tauchen_customized(rho, sigma_u, m, n, m_max,sigma_y1):
    a_bar = m * sigma_u/(1-rho**2)**0.5
    y = np.linspace(-a_bar,a_bar, n-2)
    y = np.append(-m_max * sigma_u/(1-rho**2)**0.5, y)
    y = np.append(y, m_max * sigma_u/(1-rho**2)**0.5)
    d_in = y[2] - y[1]
    d_bound = y[1] - y[0]
    
    pie = np.zeros((n,n))
    pie_inti = np.zeros(n)
    for row in range(n):
        pie[row,0]=std_norm_cdf((y[0]-rho*y[row]+d_bound/2)/sigma_u)
        pie[row,n-1]=1-std_norm_cdf((y[n-1]-rho*y[row]-d_bound/2)/sigma_u)
        
        pie[row,1]=std_norm_cdf((y[1]-rho*y[row]+d_in/2)/sigma_u)-std_norm_cdf((y[1]-rho*y[row]-d_bound/2)/sigma_u)
        pie[row,n-2]=std_norm_cdf((y[n-2]-rho*y[row]+d_bound/2)/sigma_u)-std_norm_cdf((y[n-2]-rho*y[row]-d_in/2)/sigma_u)
        
        for col in range(2,n-2):
            pie[row,col]=std_norm_cdf((y[col]-rho*y[row]+d_in/2)/sigma_u)-std_norm_cdf((y[col]-rho*y[row]-d_in/2)/sigma_u)
    
    pie_inti[0]=std_norm_cdf((y[0]+d_bound/2)/sigma_y1)
    pie_inti[n-1]=1-std_norm_cdf((y[n-1]-d_bound/2)/sigma_y1)
    
    pie_inti[1]=std_norm_cdf((y[1]+d_in/2)/sigma_y1)-std_norm_cdf((y[1]-d_bound/2)/sigma_y1)
    pie_inti[n-2]=std_norm_cdf((y[n-2]+d_bound/2)/sigma_y1)-std_norm_cdf((y[n-2]-d_in/2)/sigma_y1)
    
    for i in range(2,n-2):
        pie_inti[i]=std_norm_cdf((y[i]+d_in/2)/sigma_y1)-std_norm_cdf((y[i]-d_in/2)/sigma_y1)
    
    pie = pie / np.sum(pie, axis=1)
    pie_inti = pie_inti / np.sum(pie_inti)
    
    return y, pie, pie_inti

In [3]:
grid, pi, pi_inti = tauchen_customized(0.96,0.045**0.5,m=4,n=20,m_max=6, sigma_y1=0.38**0.5)

In [5]:
pi_inti

array([3.99655642e-10, 1.85574977e-06, 2.39158918e-05, 2.34308017e-04,
       1.65515913e-03, 8.43410144e-03, 3.10142450e-02, 8.23302138e-02,
       1.57817087e-01, 2.18489113e-01, 2.18489113e-01, 1.57817087e-01,
       8.23302138e-02, 3.10142450e-02, 8.43410144e-03, 1.65515913e-03,
       2.34308017e-04, 2.39158918e-05, 1.85574977e-06, 3.99655642e-10])

In [4]:
inc_par=solve(np.array([[20**2,20,1],[36**2,36,1],[65**2,65,1]]), np.array([0.3,1.5,0.2]))
x=np.linspace(20,98,79)
y=x**2*inc_par[0]+x*inc_par[1]+inc_par[2]
y_bar=np.clip(y,0,1.5)
y_bar[45:53]=np.linspace(0.2,0,8)

#construct mean log(y) virtually (by my eyes...)

In [5]:
e_grid=np.exp(np.tile(y_bar,(20,1)).transpose()+grid.reshape(1,20))
# 79 by 20
# mean y by age

In [6]:
β=1.011
α=0.36
δ=0.06
σ=1.5
θ=0.10
J=79
A=0.895944
j_re=46
n=0.012
surv=1-np.linspace(0,1,J+1, dtype=np.float64) ** 3.5

In [23]:
@numba.njit
def pop_dens(n, j, surv):
    dens=np.ones(j, dtype=np.float64)/j
    count=0
    err=1.
    
    while err>1e-10 and count<=2000:
        dens_new=dens.copy()
        sum=0.
        for i in range(j-1):
            dens[i+1]=dens_new[i]*surv[i+1]
            sum+=dens[i+1]
        dens[0]=n+1-sum
        dens = dens/np.sum(dens)
        
        err=np.max(np.abs(dens-dens_new))
        count+=1
        
    return dens, count

@numba.njit
def labor_dens(j, pi, pi_inti):
    count=0
    err=1.
    
    dens_lab = np.zeros((j,20), dtype=np.float64)
    dens_lab[0,] = pi_inti
    for i in range(j-1):
        dens_lab[i+1,]=dens_lab[i,]@pi
    
    dens_lab = dens_lab / np.sum(dens_lab, axis=1).reshape(-1,1)
    
    return dens_lab

In [33]:
@numba.njit
def pop_dens_new(n, j, surv):
    dens=np.zeros(j, dtype=np.float64)
    
    dens[0]=1
    for i in range(j-1):
        dens[i+1]=dens[i]*surv[i+1]/(1+n)
    
    dens = dens/np.sum(dens)
    return dens

In [8]:
μ=pop_dens_new(n,J,surv)
L_dens_temp=labor_dens(J, pi, pi_inti)
L_dens = L_dens_temp*μ.reshape(-1,1) / np.sum(L_dens_temp*μ.reshape(-1,1))

In [60]:
L=np.sum(e_grid*L_dens)

In [297]:
@numba.njit
def myclip(a, min, max):
    I, J = a.shape
    for i in range(I):
        for j in range(J):
            if a[i,j]<min:
                a[i,j]=min
            elif a[i,j]<max:
                a[i,j]=max
    return a

In [378]:
def plc_iter(r, w, t, b, θ, τ, σ, e, a_grid, surv, pi, J, j_re):
    
    inc=(1+r*(1-τ))*a_grid + (1-θ-τ)*w*e + t + b
    
    plc_con=np.ones((J,20,301), dtype=np.float64)*6.
    plc=np.ones((J,20,301), dtype=np.float64)
    plc_con[J-1,:,:]=inc[J-1,:,:]
    plc[J-1,:,:]=0
    
    for age in range(J-2,-1,-1):
        err=1.
        count=0
        while err>1e-6 and count<=500:
            itp=np.zeros((20,301), dtype=np.float64)
            plc_n=plc_con[age,:,:]
            
            for l in range(20):
                itp[l,:]=interp(a_grid[0,0,:],plc_con[age+1,l,:], inc[age,l,:]-plc_n[l,:])
            
            cons=(β*surv[age+1]* pi@(itp ** (-σ)))**(-1/σ)
            a_temp = (cons + a_grid[0,0,:]-(1-θ-τ)*w*e[age,:,:])/(1+r*(1-τ))
            
            for k in range(20):
                location = np.where((1+r*(1-τ))*a_temp[k,:]+(1-θ-τ)*w*e[age,k,:]-cons[k,:])
                if location[0].shape[0]==0:
                    sup = 0
                else:
                    sup = np.max(location[0])+1
                plc_con[age,k,sup:301] = interp(a_temp[k,:], cons[k,:], a_grid[0,0,sup:301])
                plc_con[age,k,0:sup] = (1+r*(1-τ))*a_grid[0,0,0:sup] + (1-θ-τ)*w*e[age,:,:]
        
            err=np.max(np.abs(plc_con[age,:,:],plc_n))
            count+=1
        
    return plc_con

In [317]:
@numba.njit
def asset_dens(r, w, t, b, a_grid, plc, pi_inti, pi):
    ass=np.zeros((79,20,301), dtype=np.float64)
    demo=np.zeros((79,20,301), dtype=np.float64)
    
    itv = a_grid[1:301]-a_grid[0:300]
    ass[0,:,:]=0.
    demo[0,:,0]=pi_inti

    for age in range(1,79):
        temp=np.zeros((301*20,21), dtype=np.float64)
        for e in range(20):
            for a in range(301):
                temp[a*20+e,0]=plc[age,e,a]
                temp[a*20+e,1:21]=demo[age-1,e,a] * pi[e,:]
                
        for k in range(300):
            location = np.where(np.logical_and(np.greater_equal(temp[:,0],a_grid[k]), np.less_equal(temp[:,0],a_grid[k+1])))
            l=location[0]
            demo[age,:,k] = np.sum(temp[l,1:21]*(np.reshape((temp[l,0]-a_grid[k]), (-1,1))/itv[k]), axis=0) + demo[age,:,k]
            demo[age,:,k] = np.sum(temp[l,1:21]*(np.reshape((a_grid[k+1]-temp[l,0]), (-1,1))/itv[k]), axis=0) + demo[age,:,k+1]
    
    return demo

In [318]:
a_grid=np.linspace(0,1,301, dtype=np.float64) ** 1.3 * 40
a_grid_re=a_grid.reshape(1,1,301)

In [379]:
k_min=0.
k_max=12.
t_min=0.
t_max=12

err=1.
count=0

k=8.
t=1.5

while err>1e-4 and count<=100:
    y = A * k**α * L**(1-α)
    r = A * α * (L/k)**(1-α)
    w = A * (1-α) * (k/L)**α
    τ = 0.195/(1-δ*k/y)
    b = θ*w*L/(np.sum(μ[j_re-1:]))
    
    b_vec=np.ones(J, dtype=np.float64)*b
    b_vec[0:j_re]=0
    b_vec=b_vec.reshape(-1,1,1)
    
    e=e_grid.reshape(79,20,1).repeat(301, axis=2)
    
    plc = plc_iter(r, w, t, b, e, θ, τ, σ, a_grid_re, surv, pi, J, j_re)
    plc=np.clip(plc,0,40)
    
    demo = asset_dens(r, w, t, b_vec, a_grid, plc, pi_inti, pi)
    
    ass_dist_temp=demo*μ.reshape(-1,1,1)
    ass_dist= ass_dist_temp / np.sum(ass_dist_temp)
    
    k_prime=np.sum(plc*ass_dist)
    t_prime=np.sum((1-surv[0:79]).reshape(-1,1,1) * plc * ass_dist) * ((1+r*(1-τ))) / (1+n)
    
    err=np.max(np.abs([k-k_prime,t-t_prime]))
    
    if k_prime < k:
        k_min = k
    else:
        k_max = k
    k = 0.5*(k_min+k_max)
    
    if t_prime < t:
        t_min = t
    else:
        t_max = t
    t = 0.5*(t_min+t_max)
    
    print(count)
    count+=1

D:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in power


TypeError: 'float' object is not subscriptable

In [291]:
inc=(1+r*(1-τ))*a_grid_re + (1-θ-τ)*w*e + t + b    
temp=np.zeros((20,301))
plc=np.ones((J,20,301), dtype=np.float64)*6.
plc[J-1,:,:]=0
    
for age in range(J-2,-1,-1):
    err=1.
    count=0
    while err>1e-6 and count<=500:
        itp=np.zeros((20,301), dtype=np.float64)
        plc_n=plc[age,:,:]
        for k in range(20):
            itp[k,:]=interp(a_grid_re[0,0,:],plc[age+1,k,:], plc_n[k,:])
        temp=(β*surv[age+1]* pi@((1+r*(1-τ))*plc_n + (1-θ-τ)*w*e[age,:,:] + t + b_vec[age,0,0]-itp) ** (-σ))**(-1/σ)
        plc[age,:,:]=myclip(inc[age,:,:]-temp,0,40)
        err=np.max(np.abs(plc[age,:,:],plc_n))
        count+=1

D:\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in power
  


In [284]:
temp=(β*surv[age+1]* pi@((1+r*(1-τ))*plc_n + (1-θ-τ)*w*e[age,:,:] + t + b_vec[age,0,0]-itp) ** (-σ))**(-1/σ)

In [356]:
(a_grid_re[0,0,:] - (1-θ-τ) * w * e[75,:,:]) / (1+r*(1-τ))

array([[-4.33944977e-03,  1.61408630e-02,  4.60889954e-02, ...,
         3.37102385e+01,  3.38573895e+01,  3.40046882e+01],
       [-1.97465023e-02,  7.33810435e-04,  3.06819429e-02, ...,
         3.36948314e+01,  3.38419824e+01,  3.39892811e+01],
       [-2.82050433e-02, -7.72473062e-03,  2.22234018e-02, ...,
         3.36863729e+01,  3.38335239e+01,  3.39808226e+01],
       ...,
       [-5.92755571e+00, -5.90707540e+00, -5.87712726e+00, ...,
         2.77870222e+01,  2.79341732e+01,  2.80814719e+01],
       [-8.46666226e+00, -8.44618195e+00, -8.41623381e+00, ...,
         2.52479157e+01,  2.53950667e+01,  2.55423654e+01],
       [-3.85272268e+01, -3.85067465e+01, -3.84767984e+01, ...,
        -4.81264889e+00, -4.66549790e+00, -4.51819919e+00]])

In [359]:
a_grid_re.shape

(1, 1, 301)

In [218]:
test
test

2

In [219]:
test[1]=3

In [376]:

    
inc=(1+r*(1-τ))*a_grid + (1-θ-τ)*w*e + t + b
plc_con=np.ones((J,20,301), dtype=np.float64)*6.
plc=np.ones((J,20,301), dtype=np.float64)
plc_con[J-1,:,:]=inc[J-1,:,:]
plc[J-1,:,:]=0
    
for age in range(J-2,-1,-1):
    err=1.
    count=0
    while err>1e-6 and count<=500:
        itp=np.zeros((20,301), dtype=np.float64)
        plc_n=plc_con[age,:,:]
            
        for l in range(20):
            itp[l,:]=interp(a_grid_re[0,0,:],plc_con[age+1,l,:], inc[age,l,:]-plc_n[l,:])
            
        cons=(β*surv[age+1]* pi@(itp ** (-σ)))**(-1/σ)
        a_temp = (cons + a_grid_re[0,0,:]-(1-θ-τ)*w*e[age,:,:])/(1+r*(1-τ))
            
        for k in range(20):
            location = np.where((1+r*(1-τ))*a_temp[k,:]+(1-θ-τ)*w*e[age,k,:]-cons[k,:])
            if location[0].shape[0]==0:
                sup = 0
            else:
                sup = np.max(location[0])+1
            plc_con[age,k,sup:301] = interp(a_temp[k,:], cons[k,:], a_grid_re[0,0,sup:301])
            plc_con[age,k,0:sup] = (1+r*(1-τ))*a_grid_re[0,0,0:sup] + (1-θ-τ)*w*e[age,k,:]
        
        err=np.max(np.abs(plc_con[age,:,:],plc_n))
        count+=1

array([[6.88951717e-03, 3.52209237e-02, 7.66496230e-02, ...,
        4.66458949e+01, 4.68494560e+01, 4.70532214e+01],
       [3.13504877e-02, 5.96818942e-02, 1.01110594e-01, ...,
        4.66703558e+01, 4.68739169e+01, 4.70776824e+01],
       [4.47796704e-02, 7.31110770e-02, 1.14539776e-01, ...,
        4.66837850e+01, 4.68873461e+01, 4.70911115e+01],
       ...,
       [9.41086981e+00, 9.43920121e+00, 9.48062991e+00, ...,
        5.60498752e+01, 5.62534362e+01, 5.64572017e+01],
       [1.34420763e+01, 1.34704077e+01, 1.35118364e+01, ...,
        6.00810817e+01, 6.02846428e+01, 6.04884082e+01],
       [6.11676606e+01, 6.11959920e+01, 6.12374207e+01, ...,
        1.07806666e+02, 1.08010227e+02, 1.08213992e+02]])